# Import libraries

In [1]:
import sys
from pathlib import Path
sys.path.insert(0,'..')
import glob
#!pip install openpyxl

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import h5py
import re

from functools import partial
from multiprocessing import Pool

from lhcsmapi.Time import Time
from lhcsmapi.Timer import Timer
from scipy import signal
from lhcsmapi.Time import Time
from lhcsmapi.metadata import signal_metadata
from lhcsmapi.pyedsl.dbsignal.post_mortem.PmDbRequest import PmDbRequest
from lhcsmapi.analysis.RbCircuitQuery import RbCircuitQuery

from src.utils.mp3_excel_processing import get_fgc_timestamp, get_fgc_timestamp_missing

pd.set_option('display.max_rows', 200)
import warnings
warnings.filterwarnings('ignore')

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: htt

In [3]:
date_suffix = "2022_07_07"

# Read the (clean) MP3 file

In [ ]:
mp3_fpa_df_raw = pd.read_excel(f"../data/RB_TC_extract_{date_suffix}.xlsx")
len(mp3_fpa_df_raw)

In [ ]:
# First row contains units, 9 rows contain only "Before Notebooks" and "After Notebooks" information, which need to be droped
mp3_fpa_df = mp3_fpa_df_raw.dropna(subset = ['Date (FGC)', 'Circuit Name'])
mp3_fpa_df_raw.iloc[~mp3_fpa_df_raw.index.isin(mp3_fpa_df.index)] #show droped rows

# Find FGC timestamp for each FPA event in MP3 excel

In [ ]:
mp3_fpa_df['timestamp_fgc'] = mp3_fpa_df.apply(get_fgc_timestamp, axis=1)

# Find missing fgc timestamps in mp3 file

In [ ]:
# some fgc timestamps have wrong hours
mp3_fpa_df_primary_missing = mp3_fpa_df[(mp3_fpa_df.timestamp_fgc.isna()) & (mp3_fpa_df['Nr in Q event'] == 1)]
mp3_fpa_df_primary_missing['timestamp_fgc'] = mp3_fpa_df_primary_missing.apply(get_fgc_timestamp_missing, axis=1)
found_fgc_timestamps_df = mp3_fpa_df_primary_missing["timestamp_fgc"].dropna()
mp3_fpa_df.loc[found_fgc_timestamps_df.index, "timestamp_fgc"] = mp3_fpa_df_primary_missing["timestamp_fgc"].dropna().values

In [ ]:
print(f"{len(mp3_fpa_df)} unique fgc events") # 841 Primary quenches?
print(f"{len(mp3_fpa_df[mp3_fpa_df['Nr in Q event']==1])} mp3 primary quench entries") # 834 primary quenches with correct notes

In [ ]:
mp3_fpa_df.to_csv(f"../data/RB_TC_extract_{date_suffix}_processed.csv")

# Select events to add features

In [ ]:
mp3_fpa_df = pd.read_csv("../data/RB_TC_extract_2021_11_22_processed.csv")
mp3_fpa_df_unique = mp3_fpa_df.drop_duplicates(subset=['timestamp_fgc', 'Circuit Name'])

# Time.to_string(1514761200000000000), Time.to_unix_timestamp('2018-01-01 00:00:00+01:00')
lower_limit = Time.to_unix_timestamp('2014-01-01 00:00:00+01:00')
mp3_fpa_df_period = mp3_fpa_df_unique[mp3_fpa_df_unique['timestamp_fgc'] >= lower_limit].reset_index(drop=True) 

mp3_fpa_df_to_download = mp3_fpa_df_period[mp3_fpa_df_period['Delta_t(iQPS-PIC)'].isna()]

# Add Delta_t(iQPS-PIC) timestamp

for index, row in mp3_fpa_df_to_download.iterrows():

    loc_mask = (mp3_fpa_df.timestamp_fgc == row['timestamp_fgc']) & (mp3_fpa_df["Circuit Name"] == row['Circuit Name'])

    rb_query = RbCircuitQuery(row['Circuit Family'], row['Circuit Name'], max_executions=45)
    timestamp_pic = rb_query.find_timestamp_pic(int(row['timestamp_fgc']), spark=spark)
    source_timestamp_qds_df = rb_query.find_source_timestamp_qds(int(row['timestamp_fgc']), duration=[(50, 's'), (500, 's')])

    mp3_fpa_df.loc[loc_mask, 'Delta_t(iQPS-PIC)'] = 1e-6 * (source_timestamp_qds_df['timestamp'].values - min(timestamp_pic))


mp3_fpa_df.to_csv(f"../data/RB_TC_extract_{date_suffix}_processed_filled1.csv")

# Add EE features

In [ ]:
k=0
for index, row in mp3_fpa_df_to_download.iterrows():
    try:
        loc_mask = (mp3_fpa_df.timestamp_fgc == row['timestamp_fgc']) & (mp3_fpa_df["Circuit Name"] == row['Circuit Name'])

        rb_query = RbCircuitQuery(row['Circuit Family'], row['Circuit Name'], max_executions=45)

        source_timestamp_ee_even_df = rb_query.find_source_timestamp_ee(int(row['timestamp_fgc']), system='EE_EVEN')
        timestamp_ee_even = source_timestamp_ee_even_df.timestamp[0]
        u_dump_res_even_df = rb_query.query_ee_u_dump_res_pm(timestamp_ee_even, int(row['timestamp_fgc']), system='EE_EVEN',
                                            signal_names=['U_DUMP_RES'])

        source_timestamp_ee_odd_df = rb_query.find_source_timestamp_ee(int(row['timestamp_fgc']), system='EE_ODD')
        timestamp_ee_odd = source_timestamp_ee_odd_df.timestamp[0]
        u_dump_res_odd_df = rb_query.query_ee_u_dump_res_pm(timestamp_ee_odd, int(row['timestamp_fgc']), system='EE_ODD',
                                            signal_names=['U_DUMP_RES'])

        timestamp_pic = rb_query.find_timestamp_pic(int(row['timestamp_fgc']), spark=spark)

        source_timestamp_qds_df = rb_query.find_source_timestamp_qds(int(row['timestamp_fgc']), duration=[(50, 's'), (500, 's')])
        timestamp_iQps = source_timestamp_qds_df.timestamp[0]

        # sometimes values are missing in the end
        ts_iQPS_PIC = 1e-6 * (source_timestamp_qds_df['timestamp'].values - min(timestamp_pic))
        mp3_fpa_df.loc[loc_mask, 'Delta_t(iQPS-PIC)'] = np.append(ts_iQPS_PIC, np.max(ts_iQPS_PIC)*np.ones(sum(loc_mask)-len(ts_iQPS_PIC)))

        mp3_fpa_df.loc[loc_mask, 'Delta_t(EE_odd-PIC)'] = (timestamp_ee_odd - min(timestamp_pic)) / 1e6
        mp3_fpa_df.loc[loc_mask, 'Delta_t(EE_even-PIC)'] = (timestamp_ee_even - min(timestamp_pic)) / 1e6
        mp3_fpa_df.loc[loc_mask, 'U_EE_max_ODD'] = u_dump_res_odd_df.max()[0]
        mp3_fpa_df.loc[loc_mask, 'U_EE_max_EVEN'] = u_dump_res_even_df.max()[0]
    except:
        k+=1
print(k)

In [ ]:
mp3_fpa_df.to_csv(f"../data/RB_TC_extract_{date_suffix}_processed_filled.csv")

# Reanalyze file

In [20]:
mp3_fpa_df = pd.read_csv(f"../data/RB_TC_extract_{date_suffix}_processed_filled.csv")
mp3_fpa_df_unique = mp3_fpa_df.drop_duplicates(subset=['timestamp_fgc', 'Circuit Name'])
# Time.to_string(1514761200000000000), Time.to_unix_timestamp('2018-01-01 00:00:00+01:00')
lower_limit = Time.to_unix_timestamp('2014-01-01 00:00:00+01:00')
mp3_fpa_df_period = mp3_fpa_df_unique[mp3_fpa_df_unique['timestamp_fgc'] >= lower_limit].reset_index(drop=True) 

mp3_fpa_missing_values = mp3_fpa_df_period[mp3_fpa_df_period['Delta_t(iQPS-PIC)'].isna()]
len(mp3_fpa_missing_values)#, len(mp3_fpa_df_to_download)

5

# add fpa identifier

In [31]:
mp3_fpa_df.loc[mp3_fpa_df['timestamp_fgc'].dropna().index, "fpa_identifier"] = [f"{row['Circuit Family']}_{row['Circuit Name']}_{int(row['timestamp_fgc'])}" for i, row in mp3_fpa_df.loc[mp3_fpa_df['timestamp_fgc'].dropna().index].iterrows()]
mp3_fpa_df.to_csv(f"../data/RB_TC_extract_{date_suffix}_processed_filled.csv")

# Edit Aqusition Summary

In [4]:
acquisition_summary_path = Path("../data/20220707_acquisition_summary.xlsx")
df_acquisition = pd.read_excel(acquisition_summary_path)

In [19]:
df_acquisition.to_excel(Path("../data/20220707_acquisition_summary1.xlsx"))